In [93]:
from collections import deque

class LineStabilizer2:
    """
    A class to smooth detected lines over several frames.
    """
    def __init__(self, buffer_size=5):
        # A deque is a list-like container with fast appends and pops from either end.
        self.h_buffer = deque(maxlen=buffer_size)
        self.v_buffer = deque(maxlen=buffer_size)
        
    def add_lines(self, h_lines, v_lines):
        """Adds the lines from a new frame to the buffer."""
        if h_lines is not None:
            self.h_buffer.append(h_lines)
        if v_lines is not None:
            self.v_buffer.append(v_lines)
                
    def get_h_stable_lines(self):
        """Averages the lines in the buffer to get a stable result."""
        # Combine all lines from all frames in the buffer into one big list
        all_lines = [line for frame_lines in self.h_buffer for line in frame_lines]
        
        if not all_lines:
            return None
            
        # --- We use the same merging logic as before, but on a larger, more stable dataset ---
        horizontal= []
        for line in all_lines:
            x1, y1, x2, y2 = line
            angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
            if -15 < angle < 15:
                horizontal.append(line)
        
        
        # === 1. Merge Horizontal Lines ===
        if horizontal is not None and len(horizontal) > 0:
            # Sort lines by their y-coordinate to process them from top to bottom
            horizontal.sort(key=lambda line: line[1])
            merged_lines = []
            while len(horizontal) > 0:
                base_line = horizontal.pop(0)
                y_base = (base_line[1] + base_line[3]) / 2
                x1_base = base_line[0]
                x2_base = base_line[2]
                
                group = [base_line]
                remaining_lines = []
                for line in horizontal:
                    y_curr = (line[1] + line[3]) / 2
                    if abs(y_curr - y_base) < 10 and (line[0] < x2_base + 20 and line[2] > x1_base - 20):
                        group.append(line)
                    else:
                        remaining_lines.append(line)
                horizontal = remaining_lines
                
                x_coords = np.array([line[0] for line in group] + [line[2] for line in group])
                y_coords = np.array([line[1] for line in group] + [line[3] for line in group])
                
                avg_y = int(np.mean(y_coords))
                min_x = int(np.min(x_coords))
                max_x = int(np.max(x_coords))
                
                merged_lines.append([min_x, avg_y, max_x, avg_y])
        return merged_lines
    
    def get_v_stable_lines(self):
        all_lines = [line for frame_lines in self.v_buffer for line in frame_lines]
        
        if not all_lines:
            return None
        vertical = all_lines
        if vertical is not None and len(vertical) > 0:
            # Sort lines by their x-coordinate to process them from top to bottom
            vertical.sort(key=lambda line: line[0])
            
            merged_lines = []
            while len(vertical) > 0:
                base_line = vertical.pop(0)
                group = [base_line]
                
                remaining_lines = []
                for line in vertical:
                
                    if abs(base_line[0] - line[0]) < 5 and abs(base_line[1] - line[1]) < 5 and abs(base_line[2] - line[2]) < 5 and abs(base_line[3] - line[3]) < 5:
                        group.append(line)
                    else:
                        remaining_lines.append(line)
                vertical = remaining_lines
                
                x1 = int(np.mean(np.array([line[0] for line in group])))
                x2 = int(np.mean(np.array([line[2] for line in group])))
                y1 = int(np.mean(np.array([line[1] for line in group])))
                y2 = int(np.mean(np.array([line[3] for line in group])))
                
                merged_lines.append([x1, y1, x2, y2])
        return merged_lines
                

In [94]:
class Court():
    def __init__(self):
        self.service_line_angle = 0
        self.lines = {
            'top_baseline': None,
            'bottom_baseline': None, 
            'top_service_line': None,
            'bottom_service_line': None,
            'left_singles_line': None, 
            'right_singles_line': None,
            'left_doubles_line': None,
            'right_doubles_line': None
        }


In [95]:
def classify_lines(lines):
    """
    Classify lines into horizontal and vertical based on their angle.
    """
    horizontal = []
    vertical = []
    
    for line in lines:
        x1, y1, x2, y2 = line
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        
        if -15 < angle < 15:  # Horizontal line
            horizontal.append((x1, y1, x2, y2))
        elif 60 < angle < 100 or -100 < angle < -60:  # Vertical line
            
            vertical.append((x1, y1, x2, y2))
    
    return horizontal, vertical

def _classify_lines(lines):
        """
        Classify line to vertical and horizontal lines
        """
        horizontal = []
        vertical = []
        highest_vertical_y = np.inf
        lowest_vertical_y = 0
        for line in lines:
            x1, y1, x2, y2 = line
            dx = abs(x1 - x2)
            dy = abs(y1 - y2)
            if dx > 2 * dy:
                horizontal.append(line)
            else:
                vertical.append(line)
                highest_vertical_y = min(highest_vertical_y, y1, y2)
                lowest_vertical_y = max(lowest_vertical_y, y1, y2)

        # Filter horizontal lines using vertical lines lowest and highest point
        clean_horizontal = []
        h = lowest_vertical_y - highest_vertical_y
        lowest_vertical_y += h / 15
        highest_vertical_y -= h * 2 / 15
        for line in horizontal:
            x1, y1, x2, y2 = line
            if lowest_vertical_y > y1 > highest_vertical_y and lowest_vertical_y > y1 > highest_vertical_y:
                clean_horizontal.append(line)
        return clean_horizontal, vertical

def merge_lines(horizontal):
    new_horizontal= []
    
    if horizontal is not None and len(horizontal) > 0:
        horizontal.sort(key=lambda line: line[1])
        
        while len(horizontal) > 0:
            base_line = horizontal.pop(0)
            y_base = (base_line[1] + base_line[3]) / 2
            
            group, horizontal = [base_line], [line for line in horizontal if not (abs(((line[1] + line[3]) / 2) - y_base) < 5)]
            all_x = np.array([p for line in group for p in (line[0], line[2])])
            all_y = np.array([p for line in group for p in (line[1], line[3])])
            new_horizontal.append([int(np.min(all_x)), int(np.mean(all_y)), int(np.max(all_x)), int(np.mean(all_y))])
    return new_horizontal

def detect_lines(gray):
    # Detect all lines
    lines = cv2.HoughLinesP(gray, 1, np.pi / 180, 80, minLineLength=90, maxLineGap=20)
    lines = np.squeeze(lines)
    
    unstable_horizontal, unstable_vertical = _classify_lines(lines)
    
    line_stabilizer.add_lines(unstable_horizontal, unstable_vertical)
    stable_horizontal = line_stabilizer.get_h_stable_lines()
    stable_vertical = line_stabilizer.get_v_stable_lines()
    #horizontal = merge_lines(horizontal)
    
    return stable_horizontal, stable_vertical

In [96]:
import cv2
import numpy as np

video_path = "VideoInput/video_input2.mp4"
cap = cv2.VideoCapture(video_path)

# Use your previously found source points
src_points = np.float32([[288.0, 152.0], [668.0, 150.0], [182.0, 429.0], [783.0, 428.0]])
width, height = 400, 500
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
M = cv2.getPerspectiveTransform(src_points, dst_points)

# Initialize the stabilizer before the loop
line_stabilizer = LineStabilizer2(buffer_size=15)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))
    
    # 1. Get binary image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    #edges = cv2.Canny(gray, 50, 150)
    
    horizontal, vertical = detect_lines(blurred)
    
    
    for line in horizontal:
        x1, y1, x2, y2 = line
        cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    
    
    if vertical:
        for line in vertical:
            x1, y1, x2, y2 = line
            cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
        
    #warped_frame = cv2.warpPerspective(frame, M, (width, height))
    cv2.imshow("Court Line Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 

cap.release()
cv2.destroyAllWindows()
print("Processing complete.")

Processing complete.
